In [202]:
import os

# https://sv-comp.sosy-lab.org/2025/results/results-verified/

urls = [
    "https://sv-comp.sosy-lab.org/2025/results/results-verified/META_MemSafety.table.html#/table",
    "https://sv-comp.sosy-lab.org/2025/results/results-verified/META_ReachSafety.table.html#/table",
]

def get_file(url: str, output_dir: str) -> str:
    return os.path.join(output_dir, url.split("/")[-2].replace(".html#", "") + ".json")

In [196]:
from pydantic import BaseModel, PrivateAttr, computed_field
import os
from bs4 import BeautifulSoup
from typing import List, Optional
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


class Verifier(BaseModel):
    name: str

    _verifier_name: str = PrivateAttr(default_factory=str)
    _test_date: str = PrivateAttr(default_factory=str)
    _verification_specs: list[str] = PrivateAttr(default_factory=list)

    def load_data(self) -> str:
        idx = self.name.find("[")
        prefix = self.name[:idx]
        self._verification_specs = self.name[idx+1:-1].split("; ")
        self._verifier_name = prefix[:prefix.find("2024")].strip()
        self._test_date = prefix[prefix.find("2024"):prefix.find("CET ")+3].strip()

    @computed_field
    @property
    def verifier_name(self) -> str:
        if self._verifier_name == "":
            self.load_data()
        return self._verifier_name
    
    @computed_field
    @property
    def test_date(self) -> str:
        if self._test_date == "":
            self.load_data()
        return self._test_date
    
    @computed_field
    @property
    def verification_specs(self) -> list[str]:
        if len(self._verification_specs) == 0:
            self.load_data()
        return self._verification_specs

class VerificationTask(BaseModel):
    name: str

class VerifierResult(BaseModel):
    verification_task: VerificationTask
    verifier: Verifier
    status: str
    raw_core: Optional[str]
    cpu: Optional[float]
    memory: Optional[float]

class VerificationResults(BaseModel):
    verification_results: list[VerifierResult] = []

    _verifiers_cache: List["Verifier"] = PrivateAttr(default_factory=list)
    _verification_tasks_cache: List["VerificationTask"] = PrivateAttr(default_factory=list)

    def extend(self, other:"VerificationResults") -> None:
        self.verification_results.extend(other.verification_results)
        
        self._verifiers_cache = []
        self._verification_tasks_cache = []

    @property
    def verifiers(self) -> list[Verifier]:
        if not self._verifiers_cache:
            verifiers_names = set()
            verifiers = []
            for vr in self.verification_results:
                name = vr.verifier.name
                if name not in verifiers_names:
                    verifiers.append(vr.verifier)
                    verifiers_names.add(name)
            self._verifiers_cache = verifiers
            self._verifiers_cached_names = verifiers_names
        return self._verifiers_cache
    
    @property
    def verification_tasks(self) -> list[Verifier]:
        if not self._verification_tasks_cache:
            verification_task_names = set()
            verification_tasks = []
            for vr in self.verification_results:
                name = vr.verification_task.name
                if name not in verification_task_names:
                    verification_tasks.append(vr.verifier)
                    verification_task_names.add(name)
            self._verification_tasks_cache = verification_tasks
        return self._verification_tasks_cache
    
    def summary(self, indent=0) -> str:
        text = indent * "    " + f"Verification Tasks: {len(self.verification_tasks)}\n"
        text += indent * "    " + f"Verifiers: {len(self.verifiers)}\n"
        text += indent * "    " + f"Verification Results: {len(self.verification_results)}"
        return text

In [ ]:
def get_table(soup: BeautifulSoup) -> VerificationResults:
    table = soup.find('div', class_='main-table')
    columns = get_table_headers(table)
    results = get_table_rows(table)
    verification_results = []
    
    for row in results:
        for col_i in range(len(columns)):
            i = col_i * 6 + 1

            cpu = row[i+3].replace("s", "").replace(",", ".")
            memory = row[i+4].replace("MB", "").replace(",", ".")
            verification_results.append(
                VerifierResult(
                    # i is empty
                    status=row[i+1],
                    raw_core=row[i+2],
                    cpu=float(cpu) if len(cpu)>0 else None,
                    memory=float(memory) if len(memory)>0 else None,
                    # energy i+5 is empty
                    verifier=Verifier(name=columns[col_i]),
                    verification_task=VerificationTask(name=row[0])
                )
            )

    return VerificationResults(
        verification_results=verification_results,
    )



def get_table_headers(soup: BeautifulSoup) -> list:
    table_header = soup.find('div', class_='table-header')
    headers = [header.text.strip() for header in table_header.find_all('div', class_="th header outer undefined")]
    return headers

def get_table_rows(soup: BeautifulSoup) -> list:
    table = soup.find("div", class_="table-body")
    results = []
    for row in table.find_all("div", class_="tr"):
        cells = row.find_all("div", class_="td")
        row_results = []
        for c in cells:
            row_results.append(c.text.strip())
        results.append(row_results)
    return results

In [116]:
def save_all_pages(url: str, output_dir: str = "tables", overwrite:bool=False):
    os.makedirs(output_dir, exist_ok=True)
    
    file_name = get_file(url, output_dir)
    if os.path.exists(file_name) and not overwrite:
        print(f"URL already scraped: {url}")
        return None

    driver = webdriver.Chrome()
    driver.get(url)
    wait = WebDriverWait(driver, 20)

    page_num = 1
    prev_page = None

    all_verification_results = VerificationResults()
    while True:
        # Wait for the table to load
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "main-table")))

        # Save current page
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        if prev_page == soup.prettify():
            print("No new page loaded, stopping.")
            break
        
        verification_results = get_table(soup)

        all_verification_results.extend(verification_results)

        # Try to find and click the "Next" button
        next_button = driver.find_element(By.ID, "pagination-next")
        next_class = next_button.get_attribute("class")

        # Check if "Next" is disabled
        if "disabled" in next_class.lower():
            print("Reached last page.")
            break

        # Scroll to and click the next button
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        next_button.click()

        # Optional: Wait for a status change or a brief pause to allow content to load
        time.sleep(2)
        page_num += 1
        prev_page = soup.prettify()


    driver.quit()
    
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(all_verification_results.model_dump_json(indent=2))

for url in urls:
    save_all_pages(url)

URL already scraped: https://sv-comp.sosy-lab.org/2025/results/results-verified/META_MemSafety.table.html#/table
No new page loaded, stopping.


6.0

In [197]:
from collections import defaultdict


def get_verification_results(url, output_dir="tables") -> VerificationResults:
    file_name = get_file(url, output_dir)
    if os.path.exists(file_name):
        with open(file_name, 'r', encoding='utf-8') as f:
            data = f.read()
        return VerificationResults.model_validate_json(data)
    else:
        print(f"File {file_name} does not exist.")
        return None
    
class SVCOMP:
    def __init__(self):
        self.mem_safety = get_verification_results(urls[0])
        self.reach_safety = get_verification_results(urls[1])

    def summary(self) -> str:
        return f"""SV-COMP25:
MemSafety: 
{self.mem_safety.summary(indent=1)}
ReachSafety: 
{self.reach_safety.summary(indent=1)}
"""
    
    def get_training_data(self) -> dict:
        grouped_by_verifier = defaultdict(list)

        for result in self.mem_safety.verification_results:
            key = result.verifier.verifier_name
            grouped_by_verifier[key].append(result)
        
        for result in self.reach_safety.verification_results:
            key = result.verifier.verifier_name
            grouped_by_verifier[key].append(result)

        return dict(grouped_by_verifier)
    
sv_comp = SVCOMP()

In [198]:
print(sv_comp.summary())

SV-COMP25:
MemSafety: 
    Verification Tasks: 4042
    Verifiers: 21
    Verification Results: 84882
ReachSafety: 
    Verification Tasks: 11303
    Verifiers: 26
    Verification Results: 293878



In [199]:
data = sv_comp.get_training_data()

In [200]:
len(data["2LS"])

15345

In [201]:
print(sv_comp.reach_safety.verification_results[0].model_dump_json(indent=2))

{
  "verification_task": {
    "name": "array-examples/data_structures_set_multi_proc_ground-1.ymlfalse"
  },
  "verifier": {
    "name": "2LS 2024-11-29 11:00:37 CET 2ls.[SV-COMP25_unreach-call.ReachSafety-Arrays; SV-COMP25_unreach-call.ReachSafety-BitVectors; SV-COMP25_unreach-call.ReachSafety-ControlFlow; SV-COMP25_unreach-call.ReachSafety-ECA; SV-COMP25_unreach-call.ReachSafety-Floats; SV-COMP25_unreach-call.ReachSafety-Heap; SV-COMP25_unreach-call.ReachSafety-Loops; SV-COMP25_unreach-call.ReachSafety-ProductLines; SV-COMP25_unreach-call.ReachSafety-Recursive; SV-COMP25_unreach-call.ReachSafety-Sequentialized; SV-COMP25_unreach-call.ReachSafety-XCSP; SV-COMP25_unreach-call.ReachSafety-Combinations; SV-COMP25_unreach-call.ReachSafety-Hardware; SV-COMP25_unreach-call.ReachSafety-Hardness; SV-COMP25_unreach-call.ReachSafety-Fuzzle]",
    "verifier_name": "2LS",
    "test_date": "2024-11-29 11:00:37 CET",
    "verification_specs": [
      "SV-COMP25_unreach-call.ReachSafety-Arrays",
  